# Deploying FastKafka using Docker

## Building docker image

To build a docker image for a FastKafka project we need following things,

1. A library which is built using FastKafka.
2. A file in which requirements are specified. It could be a requirements.txt file or a setup.py or it could even be a wheel file.
3. A Dockerfile to build an image which will include above two files

### Creating FastKafka code

Lets create a FastKafka based application and write to application.py file based on the [tutorial](/#tutorial)

```python
# content of the "application.py" file

from pydantic import BaseModel, NonNegativeFloat, Field

class IrisInputData(BaseModel):
    sepal_length: NonNegativeFloat = Field(
        ..., example=0.5, description="Sepal length in cm"
    )
    sepal_width: NonNegativeFloat = Field(
        ..., example=0.5, description="Sepal width in cm"
    )
    petal_length: NonNegativeFloat = Field(
        ..., example=0.5, description="Petal length in cm"
    )
    petal_width: NonNegativeFloat = Field(
        ..., example=0.5, description="Petal width in cm"
    )


class IrisPrediction(BaseModel):
    species: str = Field(..., example="setosa", description="Predicted species")

from fastkafka import FastKafka

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Iris predictions",
    kafka_brokers=kafka_brokers,
    bootstrap_servers="localhost:9092",
)

iris_species = ["setosa", "versicolor", "virginica"]

@kafka_app.consumes(topic="input_data", auto_offset_reset="latest")
async def on_input_data(msg: IrisInputData):
    global model
    species_class = model.predict([
          [msg.sepal_length, msg.sepal_width, msg.petal_length, msg.petal_width]
        ])[0]

    to_predictions(species_class)


@kafka_app.produces(topic="predictions")
def to_predictions(species_class: int) -> IrisPrediction:
    prediction = IrisPrediction(species=iris_species[species_class])
    return prediction

```

### Creating Dockerfile

```dockerfile

FROM python:3.9-slim-bullseye # (1)!

SHELL ["/bin/bash", "-c"]
WORKDIR /project

COPY application.py requirements.txt /project/

RUN pip install --no-cache-dir -r /project/requirements.txt

CMD ["fastkafka", "run", "--num-workers", "2", "application:kafka_app"]

```

1. Start from the official Python base image.


## Example repo

A `fastkafka` based library which uses above mentioned Dockerfile to build docker image can be found [here](https://github.com/airtai/sample_fastkafka_project/)